In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv


# Setup the Environment

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout
%matplotlib inline


In [5]:
train_file = '../input/digit-recognizer/train.csv'
raw_train = pd.read_csv(train_file)

In [6]:
raw_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prep data using Keras utilities

In [7]:
#Borrowed from Kaggle's Deep Learning course
img_rows, img_cols = 28, 28
num_classes = 10

def data_prep (raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)
    num_images = raw.shape[0]
    x_as_array = raw.values[:, 1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array/255 #a conventional procedure to allow color intensity to be represented by 0 to 1
    return out_x, out_y

In [8]:
X, y = data_prep(raw_train)

In [9]:
model = Sequential()
model.add(Conv2D(100, kernel_size = (3, 3), activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(100, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(100, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(100, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))#all connected layer before the prediction layer
model.add(Dense(num_classes, activation='softmax'))#this is the prediction layer

#now that the network structure is setup, let's compile it
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=3, validation_split = 0.2)

Epoch 1/3
263/263 [==============================] - 361s 1s/step - loss: 0.1708 - accuracy: 0.9471 - val_loss: 0.0585 - val_accuracy: 0.9798
Epoch 2/3
263/263 [==============================] - 368s 1s/step - loss: 0.0452 - accuracy: 0.9855 - val_loss: 0.0427 - val_accuracy: 0.9877
Epoch 3/3
263/263 [==============================] - 367s 1s/step - loss: 0.0305 - accuracy: 0.9903 - val_loss: 0.0518 - val_accuracy: 0.9833


## We have over 98% accuracy, pretty impressive, next we use our model to do the predictions and create our submission file

In [10]:
test_fp = '../input/digit-recognizer/test.csv'
test_data = pd.read_csv(test_fp)

In [11]:
test_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test_data.shape

(28000, 784)

In [13]:
x_as_array = test_data.values[:,:]
x_shaped_array = x_as_array.reshape(test_data.shape[0], img_rows, img_cols, 1)
X_test = x_shaped_array/255 #a conventional procedure to allow color intensity to be represented by 0 to 1

In [14]:
y_pred = model.predict(X_test)

In [15]:
sample_data = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [16]:
sample_data.tail()

,ImageId,Label
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0
27999,28000,0


In [17]:
y_df = pd.DataFrame(y)
y_df

,0,1,2,3,4,5,6,7,8,9
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
41995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41996,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
41998,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
y_label = np.argmax(y_pred, axis=1)

In [19]:
submission = pd.DataFrame({'ImageId': test_data.index+1, 'Label':y_label})

In [20]:
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [21]:
submission.to_csv('digit_reg_submission_noindex_v4.csv', index=False)